In [ ]:
import librosa
import librosa.display
import soundfile as sf
import numpy as np
import tempfile
import os
import shutil
from IPython.display import Audio, display
from ipywidgets import FileUpload, VBox, Output, widgets

# Create a temporary directory to store uploaded audio files
temp_dir = tempfile.TemporaryDirectory()

# Define a volume threshold
volume_threshold = 0.06  # Adjust this threshold as needed

# Create a function to classify audio based on volume
def classify_audio(audio_file):
    # Load the audio file
    audio, _ = librosa.load(audio_file, sr=None)

    # Calculate the root mean square (RMS) of the audio as a measure of volume
    rms = np.sqrt(np.mean(audio**2))

    # Classify audio based on volume threshold
    if rms > volume_threshold:
        return "happy"
    else:
        return "sad"

# Path to the output folders
output_folder_happy = "happy"
output_folder_sad = "sad"

# Create output folders if they don't exist
os.makedirs(output_folder_happy, exist_ok=True)
os.makedirs(output_folder_sad, exist_ok=True)

# Create an output widget for displaying results
output = Output()

# Create a function to handle audio classification
def handle_audio_classification(change):
    output.clear_output()

    # Get the uploaded audio file
    uploaded_file_info = upload_button.value[0]
    uploaded_file = uploaded_file_info['content']

    # Create a temporary audio file
    temp_audio_file = os.path.join(temp_dir.name, 'temp_audio.wav')

    # Save the uploaded audio to the temporary file
    with open(temp_audio_file, 'wb') as f:
        f.write(uploaded_file)

    # Classify the audio
    emotion = classify_audio(temp_audio_file)

    # Determine the output folder based on the classification
    output_path = os.path.join(output_folder_happy if emotion == "happy" else output_folder_sad, 'output_audio.wav')

    # Copy the audio file to the appropriate folder
    shutil.copy(temp_audio_file, output_path)

    with output:
        display(Audio(temp_audio_file, autoplay=True))
        print(f"Emotion: {emotion.capitalize()} - Output Path: {output_path}")

# Create a file upload widget
upload_button = FileUpload(
    accept='.wav, .mp3',  # Specify allowed file types
    multiple=False  # Allow only single file upload
)

# Attach the classification function to the upload button
upload_button.observe(handle_audio_classification, names='value')

# Display the upload button and the output widget
VBox([upload_button, output])


KeyError: ignored